In [9]:
import oracledb
import csv
from datetime import datetime

# ---- Config ----
DSN = "localhost/XEPDB1"          # service name for XE PDB
USER = "system"
PASSWORD = "12345678"

CSV_FILE = "../../data/clean_reviews.csv"

# ---- Helper: parse date safely ----
def parse_date(s):
    if not s:
        return None
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%Y/%m/%d"):
        try:
            return datetime.strptime(s.strip(), fmt).date()
        except ValueError:
            continue
    return None  # if no format matches

# ---- Main ----
def main():
    # Use thin mode (no Instant Client needed)
    with oracledb.connect(user=USER, password=PASSWORD, dsn=DSN) as conn:
        conn.autocommit = False
        cur = conn.cursor()

        # Get bank_code -> bank_id map
        cur.execute("SELECT bank_id, bank_code FROM banks")
        bank_map = {code.upper(): bid for (bid, code) in cur.fetchall()}

        rows = []
        with open(CSV_FILE, newline='', encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for r in reader:
                bank_code = (r.get("bank") or "").strip().upper()
                bank_id = bank_map.get(bank_code)
                if not bank_id:
                    print(f"⚠️ Skipping row with unknown bank: {bank_code}")
                    continue

                rows.append((
                    bank_id,
                    (r.get("source") or "Google Play").strip(),
                    None,  # source_review_id not available in your CSV
                    None,  # reviewer_name not available
                    int(r["rating"]) if r.get("rating") else None,
                    r.get("review"),
                    parse_date(r.get("date")),
                    None,  # lang_code (not in your CSV)
                    None,  # sentiment_label (not in your CSV yet)
                    None,  # sentiment_score (not in your CSV yet)
                    None,  # themes (not in your CSV yet)
                ))

        insert_sql = """
        INSERT INTO reviews (
            bank_id, source, source_review_id, reviewer_name, rating, review_text,
            review_date, lang_code, sentiment_label, sentiment_score, themes
        ) VALUES (
            :1, :2, :3, :4, :5, :6,
            :7, :8, :9, :10, :11
        )
        """

        cur.executemany(insert_sql, rows)
        conn.commit()
        print(f"✅ Inserted {len(rows)} rows into reviews.")

if __name__ == "__main__":
    main()


✅ Inserted 1480 rows into reviews.
